following the connectorx docs

In [1]:
# pip install google-cloud-bigquery
# https://github.com/googleapis/python-bigquery/blob/main/samples/query_to_arrow.py

In [2]:
import os

auth_file = "/home/ananis/notebooks/uptrain/bigquery_creds.json"
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = auth_file

In [3]:
from datetime import datetime, timedelta

import pyarrow as pa
from google.cloud import bigquery

In [9]:
def get_query(table_name, t_start, t_end):
    return """
    SELECT
        model_type,
        sig_type,
        CAST(postId AS INT64) AS postId,
        ARRAY(SELECT CAST(e AS FLOAT64) FROM UNNEST(JSON_EXTRACT_ARRAY(embs)) AS e) AS embs,
        CAST(bias AS FLOAT64) AS bias,
        tagGenre,
        IFNULL(views, 0) AS views,
        emb_update_time
    FROM {table_name}
    WHERE
        emb_len = 64
        AND tagGenre IN ('Devotion','Music & Dance','Humour & Fun','Romance & Relationships','News','Sports','Wishes')
        AND emb_update_time >= '{t_start}' 
        AND emb_update_time < '{t_end}'
    ORDER BY
        emb_update_time,
        views
    """.format(
        table_name=table_name, t_start=str(t_start), t_end=str(t_end)
    )

In [10]:
# query_str = (
#     f"SELECT COUNT(*), min(emb_update_time), max(emb_update_time) FROM {table_name}"
# )

available time  - 2023-03-11 00:11:27, 2023-03-15 23:59:59

In [11]:
def fetch_dataset(t_start, t_end):
    client = bigquery.Client()
    table_name = "`maximal-furnace-783.uptrain.ffm_embeds_reatlimeVsbatch`"
    query_str = get_query(table_name, t_start, t_end)
    query_job = client.query(query_str)
    return query_job.to_arrow()

In [12]:
%%time
INCREMENT = timedelta(minutes=1)
t_start = datetime(2023, 3, 12)
t_end = t_start + INCREMENT

arrow_table = fetch_dataset(t_start, t_end)

CPU times: user 302 ms, sys: 29.5 ms, total: 332 ms
Wall time: 5.87 s


In [13]:
len(arrow_table)

1388

## main loop

In [14]:
import concurrent.futures
import os
from datetime import datetime, timedelta

import pyarrow.parquet

In [15]:
DATASET_DIR = "./datasets/"


def get_file_name(t_start, t_end):
    return os.fspath(
        os.path.join(DATASET_DIR, f"{t_start:%Y-%m-%d-%H-%M}_{t_end:%Y-%m-%d-%H-%M}.pq")
    )


def ensure_data_exists(timepoint: datetime, window_hrs: int, batch_mins: int):
    assert 60 % batch_mins == 0
    timepoint = timepoint.replace(minute=batch_mins * (timepoint.minute // batch_mins))

    os.makedirs(DATASET_DIR, exist_ok=True)
    blocks_to_fetch = []
    dt = timepoint - timedelta(hours=window_hrs)
    while True:
        if dt >= timepoint:
            break
        dt_until = dt + timedelta(minutes=batch_mins)

        if not os.path.exists(get_file_name(dt, dt_until)):
            blocks_to_fetch.append((dt, dt_until))
        dt = dt_until
    print(f"num of blocks to fetch: {len(blocks_to_fetch)}")

    with concurrent.futures.ThreadPoolExecutor(max_workers=1) as executor:
        future_to_block = {
            executor.submit(fetch_dataset, b[0], b[1]): b for b in blocks_to_fetch
        }
        for fut in concurrent.futures.as_completed(future_to_block):
            block = future_to_block[fut]
            try:
                tbl = fut.result()
            except Exception as exc:
                print(f"Exception: {exc} raised for block: {block}")
            else:
                dt, dt_until = block
                pa.parquet.write_table(tbl, get_file_name(dt, dt_until))
                print(f"done with block: {block}")

In [16]:
TIME_NOW = datetime(2023, 3, 14, 9, 0, 0)
ensure_data_exists(TIME_NOW, 24, 15)

num of blocks to fetch: 0


## appendix

check for any active jobs

In [17]:
client = bigquery.Client()
for p in client.list_jobs(state_filter="pending"):
    print(p)
# for p in client.list_jobs(state_filter="running"):
#     print(p)

examine output

In [5]:
import duckdb

conn = duckdb.connect()
conn.execute(
    "SELECT min(emb_update_time), max(emb_update_time) FROM read_parquet('datasets/*.pq')"
)
conn.fetchall()
conn.close()